In [1]:
# 输入数据的参数
_INPUT2='{"name":"input2","type":0,"uri":"tmp_ad11481784de4a799201eb4e8a526c60"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_0e04a59c3af0450b959f61ef99c4a3bf"}'
_INPUT1='{"name":"input1","type":0,"uri":"tmp_3d44e506b1ae412199a5127ef6223600"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_54d73add97c54ee9945ce9df3ffe6fa5"},{"name":"output2","type":0,"uri":"tmp_79ccd57f088c4946a975459d1ab21119"},{"name":"output3","type":0,"uri":"tmp_51ab883d89ad4c2c91808f1f2b4dd1df"},{"name":"output4","type":0,"uri":"tmp_23babbbc267a47528e838ba6d764291d"}]'

# 自定义参数


In [2]:
# 引入使用库
import pandas as pd
import numpy as np
from ustciscrLab_A import get_score
import wfio
import random

In [3]:
# 读取测试集数据
def get_test_dataframe():
    test_dataframe = wfio.read_dataframe(_INPUT1)
    return test_dataframe

# 读取训练集 1 数据
def get_train_dataframe_1():
    train_dataframe_1 = wfio.read_dataframe(_INPUT2)
    return train_dataframe_1

# 读取训练集 2 数据
def get_train_dataframe_2():
    train_dataframe_2 = wfio.read_dataframe(_INPUT3)
    return train_dataframe_2

In [11]:
# 输入：训练集 1 (dataframe)
# 输出：自己构建的样本集 (dataframe)
def get_samples_from_train_1(train_dataframe_1):
    rows, cols = train_dataframe_1.shape
    # 定义字典，将 季度 转换为 月份，比如 q1->01，q2->04，q3->07，q4->10
    seasonToMonth = {'q1':'01','q2':'04','q3':'07','q4':'10'}
    # 高危风险行业列表
    high_risk_list = ['商贸','建材','建筑','咨询','鉴证','人力',
                    '服务','纺织','服饰','木材','中药','农产品']
    # 一共有 8 个特征，此处是 我 的 4 个特征，还有 杨文歆 的 4 个特征
    samples_from_train_1 = []
    # 获取 法人代表人姓名 重复的姓名，即有可能一个人拥有多家公司的 法人代表人姓名
    fddbrxm_ = train_dataframe_1['fddbrxm']
    fddbrxm_keep = fddbrxm_.drop_duplicates(keep='first')
    fddbrxm_no_keep = fddbrxm_.drop_duplicates(keep=False)
    fddbrxm_duplicates = fddbrxm_keep.append(fddbrxm_no_keep).drop_duplicates(keep=False)
    fddbrxm_duplicates = fddbrxm_duplicates.tolist()
    
    for i in range(rows):
        # 特征 4：入库税额 使用 is_rkse_proper 表示
        # 取 12 个季度的 入库税额，求出平均值，当平均值低于某一阈值 T 时，表示不正常
        features = []
        T = 1000
        rkse = [train_dataframe_1['2017q1rkse'][i],train_dataframe_1['2017q2rkse'][i],
                 train_dataframe_1['2017q3rkse'][i],train_dataframe_1['2017q4rkse'][i],
                 train_dataframe_1['2018q1rkse'][i],train_dataframe_1['2018q2rkse'][i],
                 train_dataframe_1['2018q3rkse'][i],train_dataframe_1['2018q4rkse'][i],
                 train_dataframe_1['2019q1rkse'][i],train_dataframe_1['2019q2rkse'][i],
                 train_dataframe_1['2019q3rkse'][i],train_dataframe_1['2019q4rkse'][i]]
        rkse = [float(x) for x in rkse] # str -> float
        ave_rkse = sum(rkse) / 12
        if ave_rkse < T:
            is_rkse_proper = 0
        else:
            is_rkse_proper = 1
        features.append(is_rkse_proper)
        
        
        numOfNull = len([x for x in rkse if x == ''])
        num0 = len([x for x in rkse if x == 0])        
        feature_rate=(numOfNull + num0)/12
        if feature_rate>0.3:
            is_rate_high = 0
        else:
            is_rate_high = 1
        features.append(is_rate_high)
        
        # 添加标签 yc
        yc = train_dataframe_1['yc'][i]
        yc = int(yc)
        features.append(yc)
        
        samples_from_train_1.append(features)
    samples_from_train_1 = np.array(samples_from_train_1)
    return samples_from_train_1



# 输入：训练集 2 (dataframe)
# 输出：自己构建的样本集 (dataframe)
def get_samples_from_train_2(train_dataframe_2):
    rows, cols = train_dataframe_2.shape
    # 定义字典，将 季度 转换为 月份，比如 q1->01，q2->04，q3->07，q4->10
    seasonToMonth = {'q1':'01','q2':'04','q3':'07','q4':'10'}
    # 高危风险行业列表
    high_risk_list = ['商贸','建材','建筑','咨询','鉴证','人力',
                    '服务','纺织','服饰','木材','中药','农产品']
    # 一共有 8 个特征，此处是 我 的 4 个特征，还有 杨文歆 的 4 个特征
    samples_from_train_2 = []
    # 获取 法人代表人姓名 重复的姓名，即有可能一个人拥有多家公司的 法人代表人姓名
    fddbrxm_ = train_dataframe_2['fddbrxm']
    fddbrxm_keep = fddbrxm_.drop_duplicates(keep='first')
    fddbrxm_no_keep = fddbrxm_.drop_duplicates(keep=False)
    fddbrxm_duplicates = fddbrxm_keep.append(fddbrxm_no_keep).drop_duplicates(keep=False)
    fddbrxm_duplicates = fddbrxm_duplicates.tolist()
    
    for i in range(rows):
        # 特征 1 ：发票是否连续多次增量增版，使用最小二乘法，使用 is_fphdsl_multi_add 表示
        # 发票核定数量
        features = []
        # 特征 4：入库税额 使用 is_rkse_proper 表示
        # 取 12 个季度的 入库税额，求出平均值，当平均值低于某一阈值 T 时，表示不正常
        T = 1000
        rkse = [train_dataframe_2['2017q1rkse'][i],train_dataframe_2['2017q2rkse'][i],
                 train_dataframe_2['2017q3rkse'][i],train_dataframe_2['2017q4rkse'][i],
                 train_dataframe_2['2018q1rkse'][i],train_dataframe_2['2018q2rkse'][i],
                 train_dataframe_2['2018q3rkse'][i],train_dataframe_2['2018q4rkse'][i],
                 train_dataframe_2['2019q1rkse'][i],train_dataframe_2['2019q2rkse'][i],
                 train_dataframe_2['2019q3rkse'][i],train_dataframe_2['2019q4rkse'][i]]
        # 检查 rkse 列表中有没有空值 ''
        numOfNull = len([x for x in rkse if x == ''])
        num0 = len([x for x in rkse if x == 0])
        if numOfNull != 0:
            for i in range(numOfNull):
                rkse.remove('')
        rkse = [float(x) for x in rkse] # str -> float
        ave_rkse = sum(rkse) / 12
        if ave_rkse < T:
            is_rkse_proper = 0
        else:
            is_rkse_proper = 1
        features.append(is_rkse_proper)
        
        
        feature_rate=(numOfNull + num0)/12
        if feature_rate>0.3:
            is_rate_high = 0
        else:
            is_rate_high = 1
        features.append(is_rate_high)
        
                # 添加标签 yc
        yc = train_dataframe_2['yc'][i]
        yc = int(yc)
        features.append(yc)
        
        
        samples_from_train_2.append(features)
    samples_from_train_2 = np.array(samples_from_train_2)
    return samples_from_train_2


# 输入：测试集 (dataframe)
# 输出：自己构建的样本集 (dataframe)
def get_samples_from_test(test_dataframe):
    rows, cols = test_dataframe.shape
    # 定义字典，将 季度 转换为 月份，比如 q1->01，q2->04，q3->07，q4->10
    seasonToMonth = {'q1':'01','q2':'04','q3':'07','q4':'10'}
    # 高危风险行业列表
    high_risk_list = ['商贸','建材','建筑','咨询','鉴证','人力',
                    '服务','纺织','服饰','木材','中药','农产品']
    # 一共有 8 个特征，此处是 我 的 4 个特征，还有 杨文歆 的 4 个特征
    samples_from_test = []
    # 获取 法人代表人姓名 重复的姓名，即有可能一个人拥有多家公司的 法人代表人姓名
    fddbrxm_ = test_dataframe['fddbrxm']
    fddbrxm_keep = fddbrxm_.drop_duplicates(keep='first')
    fddbrxm_no_keep = fddbrxm_.drop_duplicates(keep=False)
    fddbrxm_duplicates = fddbrxm_keep.append(fddbrxm_no_keep).drop_duplicates(keep=False)
    fddbrxm_duplicates = fddbrxm_duplicates.tolist()
    

    for i in range(rows):        
        # 特征 4：入库税额 使用 is_rkse_proper 表示
        # 取 12 个季度的 入库税额，求出平均值，当平均值低于某一阈值 T 时，表示不正常
        features = []
        T = 1000
        rkse = [test_dataframe['2017q1rkse'][i],test_dataframe['2017q2rkse'][i],
                 test_dataframe['2017q3rkse'][i],test_dataframe['2017q4rkse'][i],
                 test_dataframe['2018q1rkse'][i],test_dataframe['2018q2rkse'][i],
                 test_dataframe['2018q3rkse'][i],test_dataframe['2018q4rkse'][i],
                 test_dataframe['2019q1rkse'][i],test_dataframe['2019q2rkse'][i],
                 test_dataframe['2019q3rkse'][i],test_dataframe['2019q4rkse'][i]]
        # 检查 rkse 列表中有没有空值 ''
        numOfNull = len([x for x in rkse if x == ''])
        num0 = len([x for x in rkse if x == 0])
        if numOfNull != 0:
            for i in range(numOfNull):
                rkse.remove('')
        rkse = [float(x) for x in rkse] # str -> float
        ave_rkse = sum(rkse) / 16
        if ave_rkse < T:
            is_rkse_proper = 0
        else:
            is_rkse_proper = 1
        features.append(is_rkse_proper)
        
        feature_rate=(numOfNull + num0)/12
        if feature_rate>0.3:
            is_rate_high = 0
        else:
            is_rate_high = 1
        features.append(is_rate_high) 

        
        samples_from_test.append(features)
    samples_from_test = np.array(samples_from_test)
    return samples_from_test    

In [5]:
test_dataframe = get_test_dataframe()
train_dataframe_1 = get_train_dataframe_1()
train_dataframe_2 = get_train_dataframe_2()

samples_from_train_1 = get_samples_from_train_1(train_dataframe_1)
samples_from_train_2 = get_samples_from_train_2(train_dataframe_2)
samples_from_test = get_samples_from_test(test_dataframe)

In [88]:
get_score.post_user_id('17')

from sklearn.model_selection import train_test_split

data = np.vstack((samples_from_train_1,samples_from_train_2))
test_data = samples_from_test
feature = data[:,0:-1]
label = data[:,-1]
X_train,X_test,y_train,y_test = train_test_split(feature,label,
                                                 test_size=0.3,random_state=33)

from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(4, activation='relu', input_shape=(2,)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(X_test, y_test),
                    verbose=0
                   )
Probability = model.predict_proba(test_data).reshape(6000).tolist()
for i in range(len(Probability)):
    if Probability[i] > 0.2 and Probability[i] < 0.5:
        Probability[i] = Probability[i] + 0.5
    
# dabangdata = [1-x for x in dabangdata]
yc = model.predict_classes(test_data).reshape(6000).tolist()

In [89]:
prob = {
    "zjnsrsbh": test_dataframe['zjnsrsbh'],
    "Probability": Probability
}
prob = pd.DataFrame(prob)
get_score.post_verify_data(prob)

User-id:17
程序开始运行时间为：2019-12-21 01:32:08.676062
程序结束运行时间为：2019-12-21 01:32:11.548197
程序运行时间（去除打分耗时）为：2.872135
程序的准确率为：68.53377253
最终得分为：71.68039528
